## FC_1024-256_RELU

Arquitetura:
- Linear(model.fc.in_features, 1024)
- BatchNorm1d(1024)
- ReLU
- Dropout(0.4)
- Linear(1024, 256)
- BatchNorm1d(256)
- ReLU
- Dropout(0.4)
- Linear(256, 8)

Tempo de treinamento: 05:32:07

## FC_1024-64_SIGMOID
- Linear(model.fc.in_features, 1024)
- Sigmoid
- Linear(1024, 512)
- Sigmoid
- Linear(512, 64)
- Sigmoid
- Linear(64,8)

Tempo de treinamento: 05:24:21

## FC_2048-64_RELU

- Linear(model.fc.in_features, 2048)
- ReLU
- Linear(2048, 512)
- ReLU
- Linear(512, 64)
- ReLU
- Linear(64, 8)

Tempo de treinamento: 05:43:00

## FC_1024-256_SILU
- Linear(model.fc.in_features, 1024)
- nn.SiLU(),
- nn.Dropout(0.5),
- nn.Linear(1024, 256),
- nn.SiLU(),
- nn.Dropout(0.5),
- nn.Linear(256, 8)

Tempo de treinamento: 05:57:56

## FC_2048-512_GELU
- Linear(model.fc.in_features, 2048)
- GELU
- Linear(2048, 512)
- GELU
- Linear(512, 8)

Tempo de treinamento: 11:32:19

## FC_512-128_TANH

- FC_512-128_TANH:
- Linear(model.fc.in_features, 512)
- BatchNorm1d(512)
- Tanh
- Dropout(0.3)
- Linear(512, 128)
- BatchNorm1d(128)
- Tanh
- Dropout(0.3)
- Linear(128, 8)

Tempo de treinamento: 06:39:45